In [1]:
%run ../methodes/neural_network_definition.ipynb import Neural_Network_
import pandas as pd

df_train = pd.read_csv('../leaf-classification/train.csv/train.csv').set_index('id')
df_test = pd.read_csv('../leaf-classification/test.csv/test.csv').set_index('id')

In [2]:
# La constante multiplicateur (alpha) du terme de régularisation doit être > 0
alphas = np.linspace(0.000001, 0.1, 10)

In [3]:
# Initialisation des meilleurs paramètres
err_min = 100
best_alpha = -2

In [4]:
# Initialisation des tableaux pour l'analyse des données
df_alpha_train = pd.Series(0, index=alphas)
df_alpha_test = pd.Series(0, index=alphas)

In [ ]:
%%capture --no-display
from sklearn.model_selection import KFold

# Pour obtenir de bons résultats on utilise la k-fold cross-validation
kf = KFold(n_splits=10)

for alpha in alphas:
            
	# Initialisation de l'erreur moyenne
    err_train = 0
    err_test = 0
           
    # k-fold cross-validation sur les données d'entraînement
    for train, test in kf.split(df_train):
        x_train = df_train.iloc[train].drop(columns=['species'])
        t_train = df_train.iloc[train]['species']
        x_test = df_train.iloc[test].drop(columns=['species'])
        t_test = df_train.iloc[test]['species']
            
        # Initialisation du perceptron
        neural_network = Neural_Network_(alpha)

        # Entraînement
        neural_network.entrainement(x_train, t_train)

        # Calcul de l'erreur d'entraînement
        pred_train = neural_network.prediction(x_train)
        err_train += neural_network.erreur(t_train, pred_train)

        # Calcul de l'erreur de test
        pred_test = neural_network.prediction(x_test)
        err_test += neural_network.erreur(t_test, pred_test)

    # Calcul de l'erreur totale
    err_train /= 10
    err_test /= 10
    err = (err_train + err_test) / 2 

    # On garde les données en mémoire
    df_alpha_train[alpha] = err_train
    df_alpha_test[alpha] = err_test
                
    # On met à jour les meilleurs paramètres
    if err < err_min:
        err_min = err
        best_alpha = alpha

In [ ]:
import matplotlib.pyplot as plt
# Représentation graphique

# Evolution de l'erreur moyenne en fonction de alpha
plt.plot(df_alpha_train.index, df_alpha_train, color='green', label='train')
plt.plot(df_alpha_test.index, df_alpha_test, color='red', label='test')
plt.xlabel('alpha')
plt.ylabel('erreur')
plt.legend(loc="upper left")

In [ ]:
print('err_min : ', err_min, ' | alpha : ', best_alpha)